### Importações

In [1]:
import requests
from bs4 import BeautifulSoup
import warnings
import pandas as pd
import numpy as np
import time # Para adicionar um pequeno delay entre as requisições (boa prática)

### Times

In [2]:
# --- 2. CONFIGURAÇÃO E CARREGAMENTO DO HTML ---
# Desativar avisos de requisição não verificada para a URL da CBF
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

# A CBF usa o ano atual no link. Ajustei a URL para ser mais genérica.
# Se o ano for 2024, ajuste: url_cbf = "https://www.cbf.com.br/futebol-brasileiro/tabelas/campeonato-brasileiro/serie-a/2024"
url_cbf = "https://www.cbf.com.br/futebol-brasileiro/tabelas/campeonato-brasileiro/serie-a/2025" 
response = requests.get(url_cbf, verify=False)
soup = BeautifulSoup(response.content, 'html.parser')

# --- 3. EXTRAÇÃO DOS DADOS (ID e Nome) ---
# Onde guardaremos o dicionário final: {id: nome_do_time}
dicionario_times = {}

# 1. Encontrar todos os elementos que contêm as informações do time.
# O link do time está na tag <a> dentro de uma célula <td>.
links_times = soup.find_all('a', href=lambda href: href and '/futebol-brasileiro/times/' in href)

for link in links_times:
    # 2. Extrair o Nome do Time
    # O nome está na tag <strong> que é filha direta da tag <a>
    nome_tag = link.find('strong')
    if nome_tag:
        nome_do_time = nome_tag.get_text(strip=True)
    else:
        continue # Pula se não encontrar o nome (não deve ocorrer)
    
    # 3. Extrair o ID do Time
    # O ID é a última parte numérica do atributo 'href' (ex: ".../20002")
    href_completo = link.get('href')
    
    # Dividimos a string pelo caractere '/' e pegamos o último item, que é o ID
    partes_href = href_completo.split('/')
    
    # O ID é o último elemento que é um número (evitando o ano)
    id_do_time = None
    for parte in reversed(partes_href):
        if parte.isdigit():
            id_do_time = int(parte) # Converte para número inteiro
            break

    # 4. Adicionar ao dicionário
    if id_do_time and nome_do_time:
        dicionario_times[id_do_time] = nome_do_time

In [3]:
dicionario_times

{20016: 'Flamengo',
 20002: 'Palmeiras',
 59849: 'Cruzeiro Saf',
 20385: 'Mirassol',
 60175: 'Botafogo',
 20014: 'Fluminense',
 61377: 'Bahia',
 20005: 'São Paulo',
 20001: 'Corinthians',
 20013: 'Grêmio',
 60646: 'Vasco da Gama S.a.f.',
 20007: 'Red Bull Bragantino',
 62194: 'Atlético Mineiro Saf',
 20031: 'Ceará',
 20018: 'Vitória',
 20008: 'Santos Fc',
 20011: 'Internacional',
 63238: 'Fortaleza Ec Saf',
 20027: 'Juventude',
 20010: 'Sport Recife'}

### Jogos dos Times

In [4]:
# --- CONFIGURAÇÕES INICIAIS ---
warnings.filterwarnings('ignore', message='Unverified HTTPS request')
# Base URL da CBF para histórico de partidas. O ID será inserido aqui.
BASE_URL_HISTORICO = "https://www.cbf.com.br/futebol-brasileiro/times/campeonato-brasileiro/serie-a/2025/{id_time}?tab=historico-de-partidas"

# O dicionário que você gerou no passo anterior (exemplo para teste)
dicionario_times = dicionario_times

# Lista onde guardaremos o DataFrame final de cada time
all_teams_dataframes = []


## ----------------------------------------------------
## 1. FUNÇÃO PARA EXTRAIR E TRANSFORMAR DADOS DE UM TIME
## ----------------------------------------------------

def processar_historico_time(id_time, nome_time):
    """
    Busca, extrai e transforma o histórico de partidas de um time específico.
    Retorna um DataFrame processado.
    """
    print(f"Buscando e processando histórico para: {nome_time} (ID: {id_time})...")
    
    # 1. Monta a URL
    url_historico = BASE_URL_HISTORICO.format(id_time=id_time)
    
    # 2. Faz a requisição
    try:
        response_historico = requests.get(url_historico, verify=False, timeout=10)
        response_historico.raise_for_status() # Lança exceção para códigos de erro (4xx ou 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {nome_time} ({id_time}): {e}")
        return None # Retorna None se houver erro
        
    soup_historico = BeautifulSoup(response_historico.content, 'html.parser')
    
    # --- EXTRAÇÃO (Seu Código Original) ---
    partidas = soup_historico.find_all('div', class_=lambda x: x and 'styles_gameCardContainer' in x)
    dados_placares = []

    # ... (Seu código de extração do placar e nomes Time_Casa/Time_Fora) ...
    for partida in partidas:
        score_container = partida.find('div', class_=lambda x: x and 'styles_score' in x)
        
        if score_container:
            # Encontra os blocos dos dois times dentro do container de placar
            # Tentativa de ser mais robusto:
            times_scores = score_container.find_all('div', recursive=False)
            
            if len(times_scores) >= 2:
                try:
                    # Time da Casa (índice [0])
                    time_casa = times_scores[0].find('strong')['title'].strip()
                    score_casa = times_scores[0].find('span', class_=lambda x: x and 'styles_gol' in x).text.strip()
                    
                    # Time Visitante (índice [1])
                    # Nota: O índice 1 aqui é o placar do time de fora, o 2 é o nome do time de fora.
                    # A estrutura HTML deve ser: [div_time_casa, span_X, div_time_fora]
                    
                    # Reajustando a lógica de extração baseada na estrutura típica da CBF:
                    # O nome do time de fora costuma estar no mesmo nível que o de casa (strong['title']), 
                    # mas o elemento precisa ser localizado corretamente. 
                    
                    # Simplificando a extração dos blocos de time:
                    times_nomes = partida.find_all('strong')
                    scores = score_container.find_all('span', class_=lambda x: x and 'styles_gol' in x)

                    if len(times_nomes) >= 2 and len(scores) >= 2:
                        time_casa = times_nomes[0]['title'].strip()
                        score_casa = scores[0].text.strip()
                        time_fora = times_nomes[1]['title'].strip()
                        score_fora = scores[1].text.strip()
                    
                        dados_placares.append({
                            'Time_Casa': time_casa,
                            'Placar_Casa': int(score_casa),
                            'Placar_Fora': int(score_fora),
                            'Time_Fora': time_fora
                        })
                except Exception as e:
                    # print(f"Erro ao extrair placar da partida: {e}")
                    continue

    if not dados_placares:
        print(f"Nenhum dado de placar encontrado para {nome_time}.")
        return None
        
    df_partidas = pd.DataFrame(dados_placares)

    # --- TRANSFORMAÇÃO (Seu Código Original) ---
    total_jogos = len(df_partidas)
    # Garante que a ordem seja do mais antigo (1) para o mais recente (total_jogos)
    df_partidas['Ordem_Jogo'] = np.arange(total_jogos, 0, -1)
    
    NOME_TIME_FOCO = nome_time
    
    # --- Renomeando e Padronizando ---
    df_partidas['Time_Foco_ID'] = id_time # Adiciona o ID do time processado
    df_partidas['Time_Foco_Nome'] = NOME_TIME_FOCO 
    
    # Time1 (Time Foco), Time2 (Adversário)
    df_partidas['Time1'] = np.where(df_partidas['Time_Casa'] == NOME_TIME_FOCO, df_partidas['Time_Casa'], df_partidas['Time_Fora'])
    df_partidas['Time2'] = np.where(df_partidas['Time_Casa'] == NOME_TIME_FOCO, df_partidas['Time_Fora'], df_partidas['Time_Casa'])

    # Gols1 (Time Foco), Gols2 (Adversário)
    df_partidas['Gols1'] = np.where(df_partidas['Time_Casa'] == NOME_TIME_FOCO, df_partidas['Placar_Casa'], df_partidas['Placar_Fora'])
    df_partidas['Gols2'] = np.where(df_partidas['Time_Casa'] == NOME_TIME_FOCO, df_partidas['Placar_Fora'], df_partidas['Placar_Casa'])

    # Local
    df_partidas['Local'] = np.where(df_partidas['Time_Casa'] == NOME_TIME_FOCO, 'C', 'F')

    # Pontos, Saldo e Acumulados
    condicoes = [(df_partidas['Gols1'] > df_partidas['Gols2']), (df_partidas['Gols1'] == df_partidas['Gols2'])]
    pontos = [3, 1]
    
    df_partidas['Pontos_Obtidos'] = np.select(condicoes, pontos, default=0)
    df_partidas['Resultado'] = np.select(condicoes, ['V', 'E'], default='D') # Novo: V, E ou D

    df_partidas['Saldo_Gols'] = df_partidas['Gols1'] - df_partidas['Gols2']
    df_partidas['Vitoria_Obtida'] = np.where(df_partidas['Resultado'] == 'V', 1, 0)
    

    # Ordena pelo mais antigo para calcular Acumulados
    df_partidas.sort_values(by='Ordem_Jogo', ascending=True, inplace=True)
    df_partidas['Pontos_Acumulados'] = df_partidas['Pontos_Obtidos'].cumsum()
    df_partidas['Saldo_Gols_Acumulado'] = df_partidas['Saldo_Gols'].cumsum()
    df_partidas['Vitorias_Acumuladas'] = df_partidas['Vitoria_Obtida'].cumsum()
    
    # Seleciona e reordena colunas relevantes
    colunas_finais = [
        'Time_Foco_ID', 'Time_Foco_Nome', 'Ordem_Jogo', 'Local', 
        'Time1', 'Gols1', 'Gols2', 'Time2', 'Resultado',
        'Pontos_Obtidos', 'Saldo_Gols', 
        'Pontos_Acumulados', 'Saldo_Gols_Acumulado', 'Vitorias_Acumuladas'
    ]
    
    return df_partidas[colunas_finais]


## ----------------------------------------------------
## 2. ITERANDO SOBRE O DICIONÁRIO E CONSOLIDANDO OS DADOS
## ----------------------------------------------------

for id_time, nome_time in dicionario_times.items():
    df_time = processar_historico_time(id_time, nome_time)
    
    if df_time is not None:
        all_teams_dataframes.append(df_time)
        
    # Boa prática: pausa de 1 segundo para não sobrecarregar o servidor
    time.sleep(15) 

# Combina todos os DataFrames em um único
if all_teams_dataframes:
    df_analise_completo = pd.concat(all_teams_dataframes, ignore_index=True)
    
    print("\n" + "="*50)
    print("✅ Processamento de todos os times CONCLUÍDO.")
    print(f"DataFrame Final possui {len(df_analise_completo)} linhas (jogos).")
    print("Primeiras 5 linhas do DataFrame Consolidado:")
    print("="*50)
    print(df_analise_completo.head())
else:
    print("\n❌ Não foi possível processar o histórico de nenhum time.")

Buscando e processando histórico para: Flamengo (ID: 20016)...
Buscando e processando histórico para: Palmeiras (ID: 20002)...
Buscando e processando histórico para: Cruzeiro Saf (ID: 59849)...
Buscando e processando histórico para: Mirassol (ID: 20385)...
Buscando e processando histórico para: Botafogo (ID: 60175)...
Buscando e processando histórico para: Fluminense (ID: 20014)...
Buscando e processando histórico para: Bahia (ID: 61377)...
Buscando e processando histórico para: São Paulo (ID: 20005)...
Buscando e processando histórico para: Corinthians (ID: 20001)...
Buscando e processando histórico para: Grêmio (ID: 20013)...
Buscando e processando histórico para: Vasco da Gama S.a.f. (ID: 60646)...
Buscando e processando histórico para: Red Bull Bragantino (ID: 20007)...
Buscando e processando histórico para: Atlético Mineiro Saf (ID: 62194)...
Buscando e processando histórico para: Ceará (ID: 20031)...
Buscando e processando histórico para: Vitória (ID: 20018)...
Buscando e process

In [5]:
df_analise_completo.head(10)

,Time_Foco_ID,Time_Foco_Nome,Ordem_Jogo,Local,Time1,Gols1,Gols2,Time2,Resultado,Pontos_Obtidos,Saldo_Gols,Pontos_Acumulados,Saldo_Gols_Acumulado,Vitorias_Acumuladas
0,20016,Flamengo,1,C,Flamengo,1,1,Internacional,E,1,0,1,0,0
1,20016,Flamengo,2,F,Flamengo,2,1,Vitória,V,3,1,4,1,1
2,20016,Flamengo,3,F,Flamengo,2,0,Grêmio,V,3,2,7,3,2
3,20016,Flamengo,4,C,Flamengo,6,0,Juventude,V,3,6,10,9,3
4,20016,Flamengo,5,F,Flamengo,0,0,Vasco da Gama S.a.f.,E,1,0,11,9,3
5,20016,Flamengo,6,C,Flamengo,4,0,Corinthians,V,3,4,14,13,4
6,20016,Flamengo,7,F,Flamengo,1,2,Cruzeiro Saf,D,0,-1,14,12,4
7,20016,Flamengo,8,C,Flamengo,1,0,Bahia,V,3,1,17,13,5
8,20016,Flamengo,9,C,Flamengo,0,0,Botafogo,E,1,0,18,13,5
9,20016,Flamengo,10,F,Flamengo,2,0,Palmeiras,V,3,2,21,15,6


In [6]:
# Certifique-se de que o DataFrame está limpo e pronto
df = df_analise_completo.copy()

# O 'Ordem_Jogo' representa a rodada do campeonato (assumindo que 
# o jogo 1 é na Rodada 1, jogo 2 na Rodada 2, etc.)
COLUNA_RODADA = 'Ordem_Jogo'
COLUNA_TIME_ID = 'Time_Foco_ID'

# As colunas de ranqueamento acumulado
COLUNAS_ACUMULADAS = [
    'Pontos_Acumulados',     # 1º Critério
    'Vitorias_Acumuladas',   # 2º Critério
    'Saldo_Gols_Acumulado'   # 3º Critério
]

# 1. Definir o range completo das rodadas no campeonato
max_rodada = df[COLUNA_RODADA].max()
todas_rodadas = np.arange(1, max_rodada + 1)

# 2. Criar um DataFrame de esqueleto com TODAS as combinações (Time x Rodada)
times_unicos = df[COLUNA_TIME_ID].unique()
df_esqueleto = pd.MultiIndex.from_product([times_unicos, todas_rodadas], 
                                          names=[COLUNA_TIME_ID, COLUNA_RODADA]).to_frame(index=False)

# 3. Fazer o merge dos dados acumulados (apenas onde o time realmente jogou)
df_ranking_base = pd.merge(
    df_esqueleto,
    df[[COLUNA_TIME_ID, COLUNA_RODADA, 'Local', 'Time1',
       'Gols1', 'Gols2', 'Time2', 'Resultado'] + COLUNAS_ACUMULADAS].drop_duplicates(),
    on=[COLUNA_TIME_ID, COLUNA_RODADA],
    how='left'
)

# 4. PROPAGAR OS VALORES ACUMULADOS (Chave do ajuste!)
# Para cada time (groupby), preenchemos os valores nulos (NaN) com o último valor válido.
df_ranking_base[COLUNAS_ACUMULADAS] = df_ranking_base.groupby(COLUNA_TIME_ID)[COLUNAS_ACUMULADAS].ffill()

# 5. Preencher NaNs remanescentes com 0 (para times que ainda não jogaram no campeonato)
df_ranking_base[COLUNAS_ACUMULADAS] = df_ranking_base[COLUNAS_ACUMULADAS].fillna(0)

# --- 6. Calcular o Ranking (Posição) na Tabela Completa ---
# Usando os 3 critérios de ranqueamento (Pontos, Vitórias, Saldo de Gols) para desempate.

def calcular_ranking(df_grupo):
    """Função para ordenar por múltiplos critérios e aplicar o ranking."""
    # 1. Ordena o grupo (a tabela de uma rodada específica) pelos critérios.
    #    A ordem da lista/tupla (COLUNAS_ACUMULADAS) define a prioridade.
    df_ordenado = df_grupo.sort_values(
        by=COLUNAS_ACUMULADAS, 
        ascending=False  # Ordena do maior para o menor
    )
    
    # 2. Aplica o rank ao DataFrame já ordenado, usando method='first' 
    #    para garantir que os IDs de Time únicos recebam posições únicas.
    #    O método 'first' garante o desempate com base na ordem que sort_values aplicou.
    #    Em ligas de futebol, geralmente o rank vai de 1 até o número de times.
    df_ordenado['Posicao_Jogo'] = np.arange(1, len(df_ordenado) + 1)
    
    return df_ordenado

df_ranking_base = df_ranking_base.groupby(COLUNA_RODADA, group_keys=False).apply(calcular_ranking)
df_ranking_base = df_ranking_base.sort_values(by=[COLUNA_RODADA, 'Posicao_Jogo']).reset_index(drop=True)

# df_ranking_base['Posicao_Jogo'] = df_ranking_base.groupby(COLUNA_RODADA)[COLUNAS_ACUMULADAS].rank(
#     method='min',          # Empates têm o mesmo rank
#     ascending=False        # O maior valor (mais pontos) recebe a Posição 1
# ).iloc[:, 0] # Pega o ranking do 1º critério (Pontos_Acumulados)

# # 7. Merge dos Nomes e Limpeza (Opcional, mas útil)
# # Traz o nome do time de volta para a visualização
# nomes_times = df[['Time_Foco_ID', 'Time_Foco_Nome']].drop_duplicates()
# df_ranking_base = pd.merge(df_ranking_base, nomes_times, on=COLUNA_TIME_ID, how='left')


C:\Users\Alan\AppData\Local\Temp\ipykernel_17900\3230413303.py:61: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ranking_base = df_ranking_base.groupby(COLUNA_RODADA, group_keys=False).apply(calcular_ranking)


In [7]:
df_ranking_base = df_ranking_base[~df_ranking_base.Time1.isna()]

In [11]:
df_ranking_base.to_excel(r'C:\Users\Alan\Desktop\projeto_brasileirao\df.xlsx')